In [ ]:
pip install -q yahoo_fin

In [2]:
mkdir -p data results log

In [ ]:
pip install  git+https://github.com/ahsank/runml@v0.0.5#egg=runml

In [4]:
import runml.pipeline
from runml import pipeline,findata
from google.colab import drive
#from importlib import reload
# reload(runml.pipeline)

Run following if like to save results in drive

In [5]:
drive.mount('/content/drive')
!cp  /content/drive/MyDrive/colab/results/* ./results/

Mounted at /content/drive


In [6]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accuracy']/stop_loss - (1-df['Accuracy'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
pending = ['PAR']
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP',
           'BILL', 'BMBL', 'CELH', 'CFLT', 'CHGG', 'CRCT', 'CRWD',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DXCM',
           'ESTC', 'ETSY', 'EXPE', 'FOUR', 'GFS', 'GLBE', 'GOGO',
           'INDI', 'INMD', 'INTA', 'IOT','JKS', 'LUMN', 'LYFT',
           'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR',  'MXL',
           'MTCH',  'NVCR','OKTA', 'OLED', 'OSTK',
           'PERI', 'PANW', 'PAYO', 'PD', 'PLUG', 'PI', 'PINS', 'PTON', 'PUBM',
           'RBLX', 'SNAP', 'SNOW', 'SPLK', 'SPOT','STEM',
           'TEAM', 'TDOC', 'TOST', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']
lossfn = "huber_loss"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df  = pipeline.runModelCombined(tickers, 'ipos-6a', mod, False, loss=lossfn)[0]

In [ ]:
addAlloc(df, 0.10, 1)
df.sort_values('Alloc', ascending=False).head(50)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
64,U,RROnly,599.13,736.29,1335.42,23.760884,0.976562,40.46,56.759998,0.40,9.707031
53,RBLX,RROnly,387.80,578.15,965.95,24.927234,0.971429,41.15,47.240002,0.15,9.523810
39,MTCH,RROnly,801.07,817.56,1618.63,32.730726,0.962766,42.21,51.060001,0.21,9.450355
29,IOT,RROnly,107.80,59.16,166.96,9.745027,0.984848,26.26,25.459999,-0.03,9.343434
4,APP,RROnly,191.94,490.43,682.38,10.732204,0.940000,25.99,31.490000,0.21,9.114286
24,GLBE,RROnly,156.66,297.82,454.48,18.021179,0.947917,42.24,48.200001,0.14,9.107143
58,STEM,RROnly,183.96,185.64,369.60,4.399453,0.928571,6.05,8.320000,0.38,9.097744
36,MNST,RROnly,204.31,159.74,364.06,26.020930,0.962766,55.87,51.869999,-0.07,9.095745
66,UBER,RROnly,340.57,305.77,646.34,15.520273,0.952128,42.91,47.709999,0.11,9.086074
8,CFLT,RROnly,213.23,263.71,476.94,18.750489,0.944444,33.78,37.740002,0.12,8.981481


In [ ]:
findata.EPOCHS=200
later = [ ]
# https://seekingalpha.com/etfs-and-funds/etf-tables/sectors
tickers1 = ['ARKF', 'ARKK', 'ARKW', 'DAPP', 'DIA', 'DTEC', 'EEM', 'FPX',
            'ICLN', 'IJR', 'IPO', 'IXC', 'IXN', 'IXP', 'IWM', 'IYZ',
            'JETS', 'MGK', 'MGV', 'MTUM',
            'ONLN', 'QQQ', 'SMH', 'SMOG', 'SPY', 'TDIV',
            'VNQ', 'VT', 'VTI', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
# mod = pipeline.RateReturnOnly(pipeline.AddDayMonth())
df1, res = pipeline.runModelCombined(tickers1, 'etf-7a', mod, True)


In [ ]:
detail = res['ARKK'].final_df

mindate = detail['date'].min()
totalday = (detail['date'].max()-mindate).days
detail['diff'] = ((detail['date']-mindate).dt.days+1)/totalday
detail['sum_profit'] = (detail['buy_profit'] + detail['sell_profit'])/detail['adjclose']
detail['predicted_profit'] = ((detail['adjclose_15']-detail['adjclose'])/detail['adjclose']).abs()
profit_df = detail[detail['sum_profit'] > 0][['sum_profit', 'predicted_profit']]
average_profit = (profit_df['sum_profit']).mean()
excess_profit = (profit_df['sum_profit'] - profit_df['predicted_profit']).mean()
average_loss = -detail[detail['sum_profit'] < 0]['sum_profit'].mean()
detail['has_profit'] = np.where(detail['sum_profit'] > 0, 1 , 0)

accuracy = detail['has_profit'].mean()
weight_accuracy = (detail['has_profit']*detail['diff']).sum()/detail['diff'].sum()
(accuracy, weight_accuracy, average_profit, excess_profit, average_loss)

(0.9202127659574468,
 0.9229203567322334,
 0.09075827015321843,
 0.004420030125699004,
 0.01842253193096099)

In [ ]:
addAlloc(df1, 0.10, 1)
df1.sort_values('Alloc', ascending=False)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
2,ARKW,RROnly,640.17,784.96,1425.13,38.148074,0.957447,58.97,62.840000,0.07,8.966565
36,XLK,RROnly,694.51,449.24,1143.75,69.867331,0.946809,171.21,157.029999,-0.08,8.803191
11,IXC,RROnly,165.99,156.31,322.30,11.408663,0.925532,36.87,31.490000,-0.15,8.758865
7,FPX,RROnly,474.90,481.23,956.13,55.561641,0.952128,86.89,92.120003,0.06,8.723404
44,XSW,RROnly,643.16,712.68,1355.84,71.217262,0.946809,131.73,141.139999,0.07,8.708207
9,IJR,RROnly,406.81,451.00,857.80,47.690926,0.946809,98.39,104.870003,0.07,8.708207
19,MTUM,RROnly,657.53,525.42,1182.96,87.896933,0.941489,141.36,153.339996,0.08,8.683511
0,ARKF,RROnly,234.81,230.92,465.72,14.182080,0.936170,20.91,22.520000,0.08,8.563830
43,XNTK,RROnly,725.17,492.61,1217.77,64.425170,0.952128,137.80,144.240005,0.05,8.563830
22,SMH,RROnly,686.79,533.31,1220.10,51.009440,0.925532,149.05,135.130005,-0.09,8.427896


In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
# exclude = ['GTSWF', 'CBNT', 'CCL', 'CREX'. 'DTEA'. 'ILAL',
# 'TNXP', 'MRIN', 'MULN', 'AHT',
# 'LILM', 'JMIA', 'RKLB', 'VRAR', 'WISH',]
tickers3 = [ 'AIRI', 'APRN', 'AUGX', 'AUR', 'AMSC', 'ASTR', 'ATOM',
            'BBBY', 'BKKT', 'BGSF', 'BLNK', 'BSGM',
            'CARS', 'CLOV', 'CZOO',
            'DLO', 'DMTK', 'DPSI', 'EVGO', 'EXPR', 'FSLY', 'FTCH',
            'GOVX', 'GTE', 'HIMS', 'HIVE',
            'INLX',  'JOBY',  'KIND', 'KULR',
            'LAZR', 'MMMB', 'MTTR',
            'NEPH', 'NSTG', 'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST',
            'RAD', 'REAL', 'RIDE', 'RIG', 'RVYL', 'SFIX', 'STEM',
            'SFT',  'TELL', 'YEXT']
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
# mod = pipeline.RateReturnOnly(pipeline.AddDayMonth())
df3 = pipeline.runModelCombined(tickers3, 'vols-4a', mod, False)[0]

In [ ]:
# df3.sort_values('Gain', ascending=False)
addAlloc(df3, 0.17, 1)
df3.sort_values('Alloc', ascending=False).head(50)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
39,OUST,RROnly,255.96,687.94,943.90,6.203406,0.915254,4.94,6.68,0.35,5.141718
36,ONDS,RROnly,37.30,55.89,93.19,1.120860,0.900000,1.08,1.68,0.55,5.112299
22,GOVX,RROnly,27.19,38.59,65.78,0.725319,0.900000,0.65,0.86,0.33,4.991087
37,OPEN,RROnly,69.15,176.02,245.17,1.499331,0.858209,1.95,4.70,1.41,4.947727
46,STEM,RROnly,130.69,263.51,394.20,4.841042,0.906780,4.45,5.36,0.20,4.867896
40,RAD,RROnly,154.26,144.57,298.83,2.510473,0.872340,1.86,2.57,0.38,4.795468
11,BSGM,RROnly,44.65,59.65,104.30,0.513760,0.845745,1.32,2.24,0.70,4.754604
32,MTTR,RROnly,59.95,71.30,131.25,2.772340,0.872549,2.77,3.61,0.30,4.707805
18,EVGO,RROnly,61.52,94.53,156.05,4.170255,0.892857,5.72,6.76,0.18,4.656863
33,NEPH,RROnly,43.88,64.93,108.81,1.105322,0.856383,1.42,1.92,0.36,4.638611


In [ ]:
findata.EPOCHS=200
tickers4 = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DHR', 'DIS',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX', 'TMO', 'TSLA', 'TXN', 'UNH', 'UPS']
mod4 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df4 = pipeline.runModelCombined(tickers4, 'mcap-5a', mod4, False)[0]

In [ ]:
# df4.sort_values('Gain', ascending=False)
addAlloc(df4, 0.14, 1)
df4.sort_values('Alloc', ascending=False)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
23,META,RROnly,1558.47,1653.11,3211.58,93.012061,0.957447,281.00,260.070007,-0.07,6.231003
6,BA,RROnly,1801.37,1914.29,3715.66,99.792671,0.941489,219.99,205.410004,-0.07,5.889058
0,AAPL,RROnly,846.62,498.16,1344.78,50.259237,0.893617,184.92,148.440002,-0.20,5.851064
2,ADBE,RROnly,2625.92,2473.44,5099.36,269.806209,0.936170,495.18,458.630005,-0.07,5.775076
16,DIS,RROnly,671.83,921.42,1593.25,86.308430,0.904255,91.32,100.879997,0.10,5.501520
14,DE,RROnly,1767.78,826.86,2594.64,112.845246,0.872340,407.63,350.489990,-0.14,5.319149
9,CAT,RROnly,1195.74,758.82,1954.56,88.249906,0.893617,245.27,224.330002,-0.09,5.200946
20,INTC,RROnly,245.73,312.87,558.59,25.097197,0.920213,36.37,34.509998,-0.05,4.977204
33,TMO,RROnly,2076.16,1634.14,3710.30,260.128660,0.920213,537.30,562.109985,0.05,4.977204
31,QCOM,RROnly,759.40,557.26,1316.66,59.537513,0.877660,122.68,112.000000,-0.09,4.909659


In [ ]:
findata.EPOCHS=200
tickers5 = ['ABB', 'AEXAF', 'ALSMY', 'AMSSY', 'ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY', 'COTY', 'COVTY', 'CDMGF', 'CPG', 'CX',
            'DBOEY', 'DPSGY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY', 'EQNR', 'ERIC', 'ERJ','ESEA',
            'FANUY', 'FQVLF', 'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA',
            'OZON', 'RACE', 'LYB', 'LVMUY',
            'MYTE', 'NHYDY', 'NOK', 'NU', 'NXPI', 'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'ORAN', 'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'VWAPY', 'YARIY', 'ZURVY']
mod5 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df5 = pipeline.runModelCombined(tickers5, 'gstock-6b', mod5, False)[0]


In [ ]:
addAlloc(df5, 0.14)
df5.sort_values('Alloc', ascending=False).head(30)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
4,ASML,RROnly,4142.17,3231.09,7373.26,200.874222,0.946809,693.36,765.419983,0.10,5.699088
56,SE,RROnly,1367.90,1637.32,3005.22,31.756696,0.909574,61.18,51.759998,-0.15,5.291287
65,VALE,RROnly,93.15,84.58,177.73,5.203670,0.898936,14.19,11.950000,-0.16,5.157675
7,BAK,RROnly,121.64,123.77,245.41,3.765623,0.914894,10.25,9.060000,-0.12,5.116515
22,ERIC,RROnly,47.42,58.89,106.31,5.078820,0.909574,5.04,5.710000,0.13,5.105798
46,OTGLY,RROnly,85.59,128.29,213.87,4.491801,0.848837,9.89,12.560000,0.27,4.943399
6,ADYEY,RROnly,123.09,160.68,283.77,6.823201,0.905028,17.11,15.110000,-0.12,4.881617
68,YARIY,RROnly,106.57,70.99,177.56,8.430957,0.882979,19.98,16.969999,-0.15,4.746707
53,SFTBY,RROnly,164.89,199.27,364.16,13.364425,0.893617,24.69,27.910000,0.13,4.746318
35,MGA,RROnly,364.59,323.23,687.82,23.420283,0.914894,62.33,56.740002,-0.09,4.643701


In [ ]:
findata.EPOCHS=200
tickers6 = ['ARGT', 'EWD', 'CUBS', 'FLN', 'FM',
            'ECH', 'EWA', 'EWI', 'EWC', 'EWM', 'EWP', 'EWW', 'EWY', 'EWZ',
            'IDX', 'ILF', 'KEMQ', 'THD', 'TUR', 'XCEM']
mod6 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df6 = pipeline.runModelCombined(tickers6, 'cetf-7b', mod6, True)[0]


In [ ]:
addAlloc(df6, 0.10, 1)
df6.sort_values('Alloc', ascending=False)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
13,EWZ,RROnly,175.01,194.36,369.37,16.708554,0.952128,33.02,30.840000,-0.07,8.837386
0,ARGT,RROnly,207.38,129.58,336.96,14.284427,0.914894,47.43,38.869999,-0.18,8.676123
15,ILF,RROnly,104.65,133.58,238.22,12.685837,0.930851,27.73,25.879999,-0.07,8.320669
16,KEMQ,RROnly,102.41,157.14,259.55,11.917421,0.909574,15.02,16.400000,0.09,8.091017
2,CUBS,RROnly,22.14,37.86,60.00,18.708547,0.924731,19.99,20.900000,0.05,7.741935
3,FLN,RROnly,89.67,86.26,175.93,8.884696,0.930851,19.57,18.879999,-0.04,7.579787
11,EWW,RROnly,227.75,130.06,357.80,23.868301,0.904255,62.90,59.299999,-0.06,7.446809
17,THD,RROnly,237.39,290.67,528.05,45.385048,0.920213,68.46,65.800003,-0.04,7.207447
8,EWC,RROnly,153.14,67.69,220.83,16.783446,0.914894,35.80,34.529999,-0.04,7.021277
4,FM,RROnly,77.02,67.95,144.97,18.109017,0.909574,27.62,26.600000,-0.04,6.835106


In [ ]:
findata.EPOCHS=200
review = ['AMR', 'IFF', 'BLK']
tickers7 = ['AAL', 'ALB', 'APO', 'AQN', 'ARCH', 'ARE',
            'BAH', 'BAX', 'BGFV', 'BXC',
            'CBT', 'CC', 'CCJ', 'CF', 'CHK', 'CI', 'CIVI',
            'CG',  'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DVN', 'EMN',
            'F', 'FCX', 'FIS', 'FLO', 'FMS', 'GLW', 'GWW',
            'HSII', 'HPE', 'IP', 'IPI', 'IVZ', 'JWN', 'KD', 'KHC', 'KMB',
             'M', 'MMM', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK',
            'PARA', 'PCAR', 'PETS', 'PNR', 'PSTL', 'PXD', 'RCL', 'RH',
            'SAFE', 'SCCO', 'SPTN', 'SPG', 'SWK',
            'T', 'TCN', 'TGT', 'TDG', 'TWI', 'UAL', 'UBA', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW']

mod7 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df7 = pipeline.runModelCombined(tickers7, 'val-6b', mod7, False)[0]

In [8]:
addAlloc(df7, 0.10, 1)
df7.sort_values('Alloc', ascending=False).head(50)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
30,FMS,RROnly,139.51,203.56,343.07,12.876209,0.941489,26.28,28.340000,0.08,8.683511
38,JWN,RROnly,250.48,301.26,551.75,11.730722,0.925532,22.95,25.950001,0.13,8.682488
35,IP,RROnly,193.05,238.24,431.29,22.119775,0.930851,35.74,32.529999,-0.09,8.540189
56,RH,RROnly,4129.86,2681.53,6811.39,90.612399,0.941489,387.29,407.899994,0.05,8.244681
13,CF,RROnly,658.52,359.28,1017.80,20.758730,0.920213,81.23,88.070000,0.08,8.204787
69,URI,RROnly,2561.79,1531.32,4093.11,76.483470,0.882979,446.07,364.869995,-0.18,8.179669
57,SAFE,RROnly,551.31,548.51,1099.81,23.685274,0.909574,24.73,27.030001,0.09,8.091017
17,CG,RROnly,247.31,197.89,445.21,15.991252,0.936170,35.28,36.930000,0.05,8.085106
16,CIVI,RROnly,399.02,303.91,702.93,8.701244,0.904255,73.85,66.160004,-0.10,8.085106
34,HPE,RROnly,84.44,74.45,158.89,7.237447,0.920213,17.26,16.070000,-0.07,8.062310


In [ ]:
findata.EPOCHS=200
tickers8 = [ 'GLD', 'ISHG', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'UUP', 'VTIP', 'WEAT']
mod8 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df8 = pipeline.runModelCombined(tickers8, 'com-7b', mod8, True)[0]

In [10]:
addAlloc(df8, 0.10, 1)
df8.sort_values('Alloc', ascending=False)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
10,WEAT,RROnly,28.23,30.27,58.50,5.075308,0.813830,6.99,6.220000,-0.11,6.445841
1,ISHG,RROnly,80.80,90.42,171.22,63.693936,0.840426,70.42,76.279999,0.08,6.409574
8,UUP,RROnly,32.27,30.47,62.74,24.055419,0.861702,28.17,27.110001,-0.04,5.159574
4,SLV,RROnly,121.78,115.74,237.52,11.595939,0.909574,22.32,22.690001,0.02,4.574468
5,SOYB,RROnly,73.26,38.20,111.46,13.665571,0.829787,28.51,27.580000,-0.03,2.624113
0,GLD,RROnly,424.33,411.72,836.05,138.504898,0.888298,181.86,180.050003,-0.01,0.000000
3,PDBC,RROnly,39.26,37.42,76.68,6.679169,0.797872,14.74,14.750000,0.00,0.000000
6,TIP,RROnly,108.85,111.28,220.13,95.969435,0.861702,107.13,106.430000,-0.01,0.000000
7,TLT,RROnly,228.33,302.39,530.72,92.138539,0.845745,99.81,98.330002,-0.01,0.000000
9,VTIP,RROnly,27.86,14.34,42.21,41.126449,0.771277,47.26,47.080002,-0.00,0.000000


In [ ]:
findata.EPOCHS=200
tickers9 = [ 'ADA-USD', 'BTC-USD', 'DOT-USD', 'DOGE-USD',
            'ETH-USD', 'LTC-USD', 'MATIC-USD', 'XRP-USD']
mod9 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df9 = pipeline.runModelCombined(tickers9, 'cry-6b', mod9, True)[0]

In [ ]:
addAlloc(df9, 0.14, 1)
df9.sort_values('Alloc', ascending=False)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
4,ETH-USD,RROnly,25420.32,28295.52,53715.83,437.041535,0.882979,1931.92,2519.590088,0.30,5.916920
6,MATIC-USD,RROnly,14.08,19.99,34.08,0.090732,0.861702,0.66,0.900000,0.36,5.770854
0,ADA-USD,RROnly,11.40,13.08,24.48,0.152694,0.840426,0.29,0.230000,-0.20,5.205167
1,BTC-USD,RROnly,411863.10,282865.16,694728.26,12362.219550,0.840426,30462.49,36021.339844,0.18,5.116515
5,LTC-USD,RROnly,1841.21,1847.78,3689.00,51.059721,0.845745,107.40,93.040001,-0.13,4.854454
2,DOT-USD,RROnly,257.17,382.18,639.35,5.008757,0.819149,5.17,4.480000,-0.13,4.459902
3,DOGE-USD,RROnly,3.14,2.61,5.75,0.016078,0.803191,0.07,0.060000,-0.11,3.947914
7,XRP-USD,RROnly,9.31,9.41,18.72,0.256987,0.808511,0.48,0.500000,0.05,1.945289


In [ ]:
findata.EPOCHS=200
save = ['CF']
tickers10 = ['AAPL', 'AN', 'AZN', 'BJ', 'BLDR',
             'COST', 'CPRX', 'CROX', 'DHR', 'DLTR', 'DY',
             'ELF', 'EQNR',  'HRMY', 'HUM',  'LNTH',
             'MCK',  'MPWR', 'NVDA', 'ON', 'PWR', 'QQQ',  'SWAV',
             'TSLA', 'UNH', 'URI', 'VRRM', 'XLE']
mod10 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df10 = pipeline.runModelCombined(tickers10, 'trade-8b1', mod10, False)[0]

loading model from results/trade-8b1-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5


In [ ]:
addAlloc(df10, 0.07, 1)
df10.sort_values('Alloc', ascending=False)

In [ ]:
findata.G_LOOKUP_STEP = 5
mod11 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df11 = pipeline.runModelCombined(tickers10, 'trade-8b1', mod11, False)[0]


loading model from results/trade-8b1-sh-1-sc-1-sbd-0-seq-50-step-5-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5


In [ ]:
addAlloc(df11, 0.07, 1)
df11.sort_values('Alloc', ascending=False)

In [11]:
 # !cp -ru results /content/drive/MyDrive/colab
 !rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/
 ! find /content/drive/MyDrive/colab/ -type f -mtime +90 -delete -print

sending incremental file list
com-7b-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5

sent 3,221,601 bytes  received 35 bytes  6,443,272.00 bytes/sec
total size is 192,746,056  speedup is 59.83
